# Chainerチュートリアル
*http://docs.chainer.org/en/stable/tutorial/recurrentnet.html
* Recurrent Net (full backprop, truncated backprop) を学ぶ
* 様々な長さの入力時系列を扱う
* forward計算の間、ネットワークの上流を切り捨てる
  * 計算量の削減をするということか？　RNNの過去分を計算しないとか?
* ネットワーク構造を避けるために(?) 揮発性変数(Volatile Valiables)を使う
  * ?? LSTMのこと？

In [2]:
import numpy as np
import chainer
from chainer import cuda, Function, gradient_check, Variable, optimizers, serializers, utils
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L

## Recurrent Nets
* RecurrentNet とはloopを持ったニューラルネットワークで、入力シーケンス$x_1, x_2, \cdots, x_t$と初期状態$h_0$を与えられる。そして、$h_t = f\left( x_t, h_{t-1} \right)$ によって繰り返し計算する。
* 出力は$y_t = \right( h_t \left)$
* この手順を拡大すると、通常のフィードフォワードネットワークのように解釈できる
* 
* シンプルなRNNの例として、１層のlanguage-modelを学ぶ
* これは、有限な単語列を与えられ、続く単語を予測したい。
* 1000の異なる単語を想定しており、それぞれの単語は１００次元のベクトルで表現される
* 
* ニューラルネット言語モデルにLSTM(chainer.links.LSTM)が使える。
* このlinkは、普通のfully-connected層のように見える。宣言時に、入力と出力のサイズを設定する


In [3]:
l = L.LSTM(100, 50)

* forward計算を行う前に、LSTM層のリセットをする必要がある
* LSTM層には内部状態を持っている（過去の状態など）

In [4]:
l.reset_state()
x = Variable(np.random.randn(10, 100).astype(np.float32))
y = l(x)

* 次のステップからの入力は直接LSTM層に入力できる

In [6]:
x2 = Variable(np.random.randn(10, 100).astype(np.float32))
y2 = l(x2)

* LSTMを使ってRNNを書いてみる
*
* EmbedIDは一致する固定次元のembedding vectorに入力整数を変換する

In [7]:
class RNN(Chain):
    def __init__(self):
        super(RNN, self).__init__(
            embed=L.EmbedID(1000, 100),  # word embedding
            mid=L.LSTM(100, 50),  # the first LSTM layer
            out=L.Linear(50, 1000),  # the feed-forward output layer
        )

    def reset_state(self):
        self.mid.reset_state()

    def __call__(self, cur_word):
        # Given the current word ID, predict the next word.
        x = self.embed(cur_word)
        h = self.mid(x)
        y = self.out(h)
        return y

rnn = RNN()
model = L.Classifier(rnn)
optimizer = optimizers.SGD()
optimizer.setup(model)

* word変数のリストx_listを使って、シンプルなforループで損失を計算できる

In [8]:
def compute_loss(x_list):
    loss = 0
    for cur_word, next_word in zip(x_list, x_list[1:]):
        loss += model(cur_word, next_word)
    return loss

* 上記の積み重ねられた損失変数には、過去の計算の履歴が含まれている
* backward()メソッドを使ってモデルパラメータの勾配を計算できる

In [9]:
# Suppose we have a list of word variables x_list.
rnn.reset_state()
model.zerograds()
loss = compute_loss(x_list)
loss.backward()
optimizer.update()

NameError: name 'x_list' is not defined

* データx_listを入力することでシーケンスを扱える

## Truncate the Graph by Unchaining
* RNNの用途として、非常に長いシーケンスを扱いたい場合がある. 
* メモリに収まらないほど長いシーケンスの場合、短い時間幅にbackpropagationをきりつめる
* これを"truncated backprop"と呼んでいる
* この手法はヒューリスティックであり、勾配の偏りを生む。しかし、とても長いシーケンスを扱う場合によく機能する
* Chainerでは、"backward unchaining"と呼ぶ仕組みで、簡単に"truncated backprop"を実装できる
* Variableオブジェクトを使い計算を始める。自動的にシーケンスの切り分けが実施される
* 結果として、長くない計算履歴を持つ
* 
* 上記のRNNと同じネットワークで"backward unchain"を使って描く
* とても長いシーケンスに対して、３０ステップ毎に"truncated backprop"させる

In [ ]:
loss = 0
count = 0
seqlen = len(x_list[1:])

rnn.reset_state()
for cur_word, next_word in zip(x_list, x_list[1:]):
    loss += model(cur_word, next_word)
    count += 1
    if count % 30 == 0 or count == seqlen:
        model.zerograds()
        loss.backward()
        loss.unchain_backward()
        optimizer.update()

* model()でStateがアップデートされ、lossに損失が蓄積される
* 累積損失から後方へ計算履歴を削除する時にunchain_backwardメソッドが呼ばれる